In [1]:
import numpy as np

In [73]:
def step(x, lamb, gradf, hessianf, h, gradh, hessianh, stepsize=1):
    """
    :param x:
    :param lamb:
    :param gradf:
    :param hessianf:
    :param h:
    :param gradh:
    :param hessianh:
    :param stepsize:
    :return:
    """

    hessianLagrangian = hessianf(x) - lamb*hessianh(x)
    gradh = gradh(x).reshape((gradh(x).shape[0],1))


    topPart = np.concatenate((hessianLagrangian, -gradh), axis=1)
    bottomPart = np.concatenate((-gradh.T, np.zeros((gradh.shape[1], gradh.shape[1]))), axis=1)

    mat = np.concatenate((topPart, bottomPart))
    rhs = np.concatenate((-gradf(x)-lamb*gradh.T, h(x)), axis=1).reshape((3,1))


    res = np.linalg.solve(mat, rhs) #np.linalg.inv(mat) @ rhs

    xdir = res[0:len(res)-1].reshape(x.shape)
    lambdir = res[-1]
    return x + xdir, lamb + lambdir


In [23]:
import math
import numpy as np

def gradf(x):
    return np.array([ 3*(math.e)**(3*x[0]), -4**(x[1])*math.e**(-4*x[1])*math.log(4, math.e)])

def gradh(x):
    return np.array([2*x[0],2*x[1]])

def h(x):
    return np.array([x[0]**2 + x[1]**2 - 1]).reshape((1,1))

def hessianh(x):
    return np.array([2,0,0,2]).reshape(2,2)

def hessianf(x):
    return np.array([9*math.e**(3*x[0])*math.log(math.e,math.e)**2,0,0,16*math.e**(-4*x[1])]).reshape((2,2))


In [58]:
# Try our Step function with the given example

x, l = step(np.array([-1, 1]), -1, gradf, hessianf, h, gradh, hessianh)
print(x,l)

(3, 1) [-0.26825641] (2, 2)
xdir [ 0.23174359 -0.26825641]
[-0.76825641  0.73174359] [-0.35834444]


In [82]:
def optimise(x_start, lamb_start, gradf, hessianf, h, gradh, hessianh, steps=100, stepsize=1):
    x, lamb = x_start, lamb_start
    for i in range(steps):
        x, lamb = step(x, lamb, gradf, hessianf, h, gradh, hessianh)
    return x

In [84]:
x = np.array([-1, 1])
lamb = -1
steps = 1000
print(optimise(x, lamb, gradf, hessianf, h, gradh, hessianh, steps))


[-0.7379972   0.67480377]


In [ ]:
startingPoints = [[[x],[lam]], ]

for x,lamb in startingPoints:
    print(optimise(x, lamb, gradf, hessianf, h, gradh, hessianh, steps))
